# Vgrid DGGS key features

You can try out vgrid by using the cloud-computing platforms below without having to install anything on your computer:

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/00_intro.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

For working with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/), [vgridpandas document](https://vgridpandas.gishub.vn)

## Install vgrid

In [ ]:
# %pip install vgrid --upgrade

## DGGS Conversion
### latlon2dggs

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2h3

lat = 10.775276
lon = 106.706797
res = 10
h3_id = latlon2h3(lat, lon, res)
h3_id

### DGGS to Shapely Polygons

In [ ]:
import geopandas as gpd
from vgrid.conversion.dggs2geo.h32geo import h32geo
h3_geo = h32geo([h3_id])
h3_geo

### DGGS to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geojson

h3_geojson = h32geojson(h3_id)
print(h3_geojson)

### CSV to DGGS

In [ ]:
import pandas as pd
from vgrid.conversion.csv2dggs import csv2s2
csv_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/point.csv"
csv_file = pd.read_csv(
    csv_path
)  # modify the path to the point.csv file containing DGGS cell Ids
s2_geojson = csv2s2(csv_path, "s2")  # csv2s2(csv_path) by default for 's2' column
s2_geojson

### Vector to DGGS

In [ ]:
from vgrid.conversion.vector2dggs.vector2rhealpix import vector2rhealpix

# Read GeoJSON from URL
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)

geojson_rhealpix = vector2rhealpix(file_path, resolution=10, compact=False)
# Change compact to True to see the compact version of the rHEALPix cells

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpix_layer = folium.GeoJson(
    geojson_rhealpix,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["rhealpix", "resolution", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpix_layer.get_bounds())

# Display the map
m

### DGGS Compact

In [ ]:
from vgrid.conversion.dggscompact import rhealpixcompact
rhealpix_compacted = rhealpixcompact(geojson_rhealpix, rhealpix_id="rhealpix")

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpixcompacted_layer = folium.GeoJson(
    rhealpix_compacted,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["rhealpix", "resolution", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpixcompacted_layer.get_bounds())

# Display the map
m

#### DGGS Expand

In [ ]:
from vgrid.conversion.dggscompact import rhealpixexpand
rhealpix_expanded = rhealpixexpand(geojson_rhealpix, resolution=11, rhealpix_id="rhealpix")

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpixexpanded_layer = folium.GeoJson(
    rhealpix_expanded,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["rhealpix", "resolution", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpixexpanded_layer.get_bounds())

# Display the map
m

### Raster to DGGS

In [ ]:
from vgrid.conversion.raster2dggs.raster2tilecode import raster2tilecode
from vgrid.utils.download import download_file

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

# Convert rasteto GeoJSON raster2h3(raster_file)
raster2isea4t_csv = raster2tilecode(raster_file, output_format="csv")
# print(raster2isea4t_csv)
raster2isea4t_geojson = raster2tilecode(raster_file, output_format="geojson")

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

h3_layer = folium.GeoJson(
    raster2isea4t_geojson,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["tilecode", "resolution", "band_1", "band_2", "band_3", "cell_area"],
        aliases=["H3 ID", "Resolution", "Band 1", "Band 2", "Band 3", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(h3_layer.get_bounds())

# Display the map
m

## DGGS Generator

In [ ]:
from vgrid.generator.h3grid import h3grid

h3_grid = h3grid(resolution=0,output_format="gpd")
h3_grid.plot(edgecolor='white')

### DGGS Statistics

In [ ]:
from vgrid.stats.h3stats import h3stats
h3stats()